In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, SubsetRandomSampler
import numpy as np
  

In [3]:
batch_size = 256
dataset_dir = 'fruits-360'

# Set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Normalize the input data using ImageNet statistics
imagenet_mean = [0.485, 0.456, 0.406]
imagenet_std = [0.229, 0.224, 0.225]

In [4]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(imagenet_mean, imagenet_std)
])

# Load the datasets
train_dataset = torchvision.datasets.ImageFolder(
    root=f'{dataset_dir}/train',
    transform=transform
)
val_dataset = torchvision.datasets.ImageFolder(
    root=f'{dataset_dir}/val',
    transform=transform
)
test_dataset = torchvision.datasets.ImageFolder(
    root=f'{dataset_dir}/test',
    transform=transform
)

# Create data loaders
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
train_loader = torch.utils.data.DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
val_loader = torch.utils.data.DataLoader(val_dataset, shuffle=True, batch_size=batch_size)

In [5]:
# Initialize the ResNet-18 model
model = torchvision.models.vgg16(pretrained=False)
num_classes = len(train_dataset.classes)

# Freeze the parameters of all layers
for param in model.parameters():
    param.requires_grad = False

# Replace the last fully connected layer with a new one
num_features = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_features, num_classes)
model.to(device)
model.to(device)

print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [ ]:
# Training loop
num_epochs = 50
min_loss = np.inf
for epoch in range(num_epochs):
    print(f'training... epoch {epoch}')
    running_loss = 0.0
    val_loss = 0.0
    model.train()
    for i, (inputs, labels) in enumerate(train_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    model.eval()   
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            vloss = criterion(outputs, labels)
            val_loss += vloss.item()
        
    if val_loss < min_loss:
        min_loss = val_loss
        torch.save(model, 'model.pth')
        print(f'saving model at epoch {epoch}')
        print(f'Epoch {epoch + 1}, Batch {i + 1}: loss {running_loss / 200:.3f} val_loss {val_loss / 200:.3f}')
            

print('Training finished!')



In [ ]:
# Evaluation on the test set
model.eval()  # Switch to evaluation mode
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Test Accuracy: {accuracy:.2f}%')
